# **Agentic RAG**

Authored by [Kalyan KS](https://www.linkedin.com/in/kalyanksnlp/). To stay updated with LLM, RAG and Agent updates, you can follow me on [Twitter](https://x.com/kalyan_kpl).


1. **Rewrite Agent**: Rewrites the user query if required.

2. **Router Agent**: Analyzes the user query and returns "vector_store" if the query is related to Akmmus AI Labs, "web_search" otherwise.

3. **Retriever Agent**: Depending on the output of router agent, uses website search tool (vector store) or DuckDuckGoSearch tool and returns the relevant context.

4. **Evaluator Agent**: Analyzes the query and context, returns "yes" if the context is relevant and contains the answer, "no" otherwise.

5. **Answer Agent**: Based on evaluator's output, generates a clear and concise answer if "yes", returns "Unable to answer the given query" if "no"

## **Install and import libraries**

In [ ]:
!pip install crewai crewai-tools langchain-community duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of duckduckgo-search to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.4 MB/s eta 0:00:00

In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import WebsiteSearchTool
from langchain_community.tools import DuckDuckGoSearchResults

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


## **Set up the LLM API Key**

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

## **Set up Tools**

In [ ]:
# Set up website search tool
website_search_tool = WebsiteSearchTool(website='https://www.akmmusai.pro/',
    config=dict(
        llm=dict(
            provider="openai",
            config=dict(
                model="gpt-4o-mini",
            ),
        ),
        embedder=dict(
            provider="openai",
            config=dict(
                model="text-embedding-3-small",
            ),
        ),
    )
)

In [ ]:
website_search_tool.run("What is Akmmus AI Labs?")

Using Tool: Search in a specific website


'Relevant Content:\nAkmmus AI | NLP-focused AI research lab top of pageNLP-focused AI Research LabTirupati, IndiaAkmmus AI is a newly established NLP-focused AI research lab from Tirupati, India. Our broad areas of research interest are natural language processing and deep learning. Specifically, we are interested in large language models and the relevant topics.Katikapalli Subramanyam KalyanNLP Researcher (Founder)Research InterestsLarge Language ModelsPrompt Engineering for LLMsLLM RobustnessSynthetic Text Data Generation using LLMsApplications of LLMs to NLU and NLG TasksLatest News Akmmus AI website is launched on September 23, 2023. "A Survey of GPT-3 Family Large Language Models including ChatGPT and GPT-4" is available online. (Paper Link) bottom of page'

In [ ]:
# Web Search Tool
from typing import Type, Any
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.tools import DuckDuckGoSearchResults

class DuckDuckGoSearchInput(BaseModel):
    """Input schema for DuckDuckGoSearch tool."""
    query: str = Field(..., description="The search query to use with DuckDuckGo.")

class DuckDuckGoSearchTool(BaseTool):
    name: str = "DuckDuckGo Search"
    description: str = "Searches the web using DuckDuckGo for the provided query."
    args_schema: Type[BaseModel] = DuckDuckGoSearchInput

    def _run(self, query: str) -> Any:
        search = DuckDuckGoSearchResults(output_format="list")
        results = search.run(query)
        return results

In [ ]:
web_search_tool = DuckDuckGoSearchTool()
web_search_tool.run("What is ChatGPT?")

Using Tool: DuckDuckGo Search


[{'snippet': 'ChatGPT is a free AI chatbot that can answer questions, assist with various tasks, and generate text, code, and more. Learn how to access, use, and benefit from ChatGPT, and what are the concerns and limitations of this generative AI tool.',
  'title': "What is ChatGPT? How the world's most popular AI chatbot can benefit ...",
  'link': 'https://www.zdnet.com/article/what-is-chatgpt-how-the-worlds-most-popular-ai-chatbot-can-benefit-you/'},
 {'snippet': 'ChatGPT is an AI model that uses deep learning to generate human-like text based on prompts from users like you. It works by predicting the next word in a given text, based on the patterns it has learned from a massive amount of data during its training process.',
  'title': 'What Is ChatGPT and How Does It Work? - How-To Geek',
  'link': 'https://www.howtogeek.com/871071/what-is-chatgpt/'},
 {'snippet': 'ChatGPT is a generative AI chatbot that can communicate with users and generate text, code, images and more. Learn how

## **Define Agents**

In [ ]:
# Set up LLM
llm = LLM(model="gpt-4o-mini")

In [ ]:
# Rewrite Agent
rewrite_agent = Agent(
    role="Query Rewriting Specialist",
    goal="Analyze and improve search query for better results",
    backstory="""Expert in natural language processing and query optimization
    with years of experience in improving search effectiveness.""",
    llm=llm,
    allow_delegation=False,
    verbose=True
)


In [ ]:
# Router Agent
router_agent = Agent(
    role="Traffic Router",
    goal="Determine the appropriate search destination based on query content",
    backstory="""Specialized in query analysis and routing with deep understanding
    of various data sources and their applicability.""",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [ ]:
# Retriever Agent
retriever_agent = Agent(
    role="Information Retriever",
    goal="Fetch relevant information from appropriate source",
    backstory="""Experienced in efficient information retrieval from multiple
    sources with expertise in both web and vector store searches.""",
    llm=llm,
    tools=[website_search_tool, web_search_tool],
    allow_delegation=False,
    verbose=True
)


In [ ]:
# Evaluator Agent
evaluator_agent = Agent(
    role="Content Evaluator",
    goal="Assess the relevance and completeness of retrieved information",
    backstory="""Expert in content analysis and quality assessment with
    strong analytical skills and attention to detail.""",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [ ]:
# Answer Agent
answer_agent = Agent(
    role="Response Generator",
    goal="Generate clear and accurate answers",
    backstory="""Skilled in synthesizing information and crafting clear,
    concise responses while maintaining accuracy and relevance.""",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## **Define Tasks**

In [ ]:
# Task for Rewrite Agent
rewrite_task = Task(
    description="""Analyze the input query: {query}, and rewrite it if needed to improve search effectiveness.
    Keep the query as is if it's already well-formed. Consider adding relevant keywords
    and removing unnecessary words while maintaining the original intent.""",
    expected_output="""A well-formed query that maintains the original intent but is optimized
    for search. Return the original query if no rewriting is needed.""",
    agent=rewrite_agent
)

In [ ]:
# Task for Router Agent
router_task = Task(
    description="""Analyze the query to determine if it's related to Akmmus AI Labs.
    Return 'vector_store' if the query is about Akmmus AI Labs, otherwise return 'web_search'.
    Look for keywords related to Akmmus, AI Labs, or specific projects/products.""",
    expected_output="""A string containing either 'vector_store' or 'web_search' based on
    the query analysis. The output should be lowercase and match exactly one of these two options.""",
    agent=router_agent,
    context=[rewrite_task]
)

In [ ]:
# Task for Retriever Agent
retriever_task = Task(
    description="""Use the appropriate search tool based on the router's output to fetch relevant information.
    For 'vector_store', search Website search tool. For 'web_search', use web search tool.""",
    expected_output="""A string containing the search results. The output should include relevant
    information that directly addresses the query, with sources when available.""",
    agent=retriever_agent,
    context=[rewrite_task, router_task]
)

In [ ]:
# Task for Evaluator Agent
evaluator_task = Task(
    description="""Analyze the retrieved information to determine if it's relevant to the query,
    and contains a sufficient answer. Consider factors like content relevance, completeness,
    and reliability of the information.""",
    expected_output="""A string containing 'yes' if the content is relevant and contains
    a sufficient answer, or 'no' if the content is irrelevant or insufficient.""",
    agent=evaluator_agent,
    context=[rewrite_task, retriever_task]
)

In [ ]:
# Task for Answer Agent
answer_task = Task(
    description="""Generate a comprehensive answer to the query based on the retrieved information.
    If the evaluator returned 'yes', synthesize the information into a clear and concise response.
    If 'no', return 'Unable to answer the given query'.""",
    expected_output="""A string containing either a well-formatted answer based on the retrieved
    information or 'Unable to answer the given query' if the information was deemed insufficient.""",
    agent=answer_agent,
    context=[rewrite_task, retriever_task, evaluator_task]
)

## **Define Crew**

In [ ]:
agentic_rag_crew = Crew(
    agents=[rewrite_agent, router_agent, retriever_agent, evaluator_agent, answer_agent],
    tasks=[rewrite_task, router_task, retriever_task, evaluator_task, answer_task],
    verbose=True,
)

## **Run Crew**

In [ ]:
inputs ={"query":"Who is the founder of Akmmus AI Labs?"}

In [ ]:
result = agentic_rag_crew.kickoff(inputs=inputs)

# Agent: Query Rewriting Specialist
## Task: Analyze the input query: Who is the founder of Akmmus AI Labs?, and rewrite it if needed to improve search effectiveness. 
    Keep the query as is if it's already well-formed. Consider adding relevant keywords 
    and removing unnecessary words while maintaining the original intent.


# Agent: Query Rewriting Specialist
## Final Answer: 
Who founded Akmmus AI Labs?


# Agent: Traffic Router
## Task: Analyze the query to determine if it's related to Akmmus AI Labs. 
    Return 'vector_store' if the query is about Akmmus AI Labs, otherwise return 'web_search'. 
    Look for keywords related to Akmmus, AI Labs, or specific projects/products.


# Agent: Traffic Router
## Final Answer: 
vector_store


# Agent: Information Retriever
## Task: Use the appropriate search tool based on the router's output to fetch relevant information. 
    For 'vector_store', search Website search tool. For 'web_search', use web search tool.




# Agent: Information Retriever
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Who founded Akmmus AI Labs?\"}"
## Tool Output: 
Relevant Content:
Akmmus AI | NLP-focused AI research lab top of pageNLP-focused AI Research LabTirupati, IndiaAkmmus AI is a newly established NLP-focused AI research lab from Tirupati, India. Our broad areas of research interest are natural language processing and deep learning. Specifically, we are interested in large language models and the relevant topics.Katikapalli Subramanyam KalyanNLP Researcher (Founder)Research InterestsLarge Language ModelsPrompt Engineering for LLMsLLM RobustnessSynthetic Text Data Generation using LLMsApplications of LLMs to NLU and NLG TasksLatest News Akmmus AI website is launched on September 23, 2023. "A Survey of GPT-3 Family Large Language Models including ChatGPT and GPT-4" is available online. (Paper Link) bottom of page


# Agent: Information Retriever
## Final Answer: 
Akmmus AI Labs 

In [ ]:
print(result)

Akmmus AI Labs was founded by Katikapalli Subramanyam Kalyan, who is an NLP (Natural Language Processing) researcher. The lab specializes in natural language processing and deep learning, with a particular focus on large language models and related subjects.
